In [1]:
# %%shell
# # Ubuntu no longer distributes chromium-browser outside of snap
# # Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# # Add Debian Buster repository
# cat > /etc/apt/sources.list.d/debian.list << "EOF"
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
# EOF

# # Add keys
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

# # Export keys to keyrings
# apt-key export 77E11517 | gpg --dearmor -o /usr/share/keyrings/debian-buster.gpg
# apt-key export 22F3D138 | gpg --dearmor -o /usr/share/keyrings/debian-buster-updates.gpg
# apt-key export E562B32A | gpg --dearmor -o /usr/share/keyrings/debian-security-buster.gpg

# # Prefer Debian repository for chromium* packages only
# # Note the double-blank lines between entries
# cat > /etc/apt/preferences.d/chromium.pref << "EOF"
# Package: *
# Pin: release a=eoan
# Pin-Priority: 500


# Package: *
# Pin: origin "deb.debian.org"
# Pin-Priority: 300


# Package: chromium*
# Pin: origin "deb.debian.org"
# Pin-Priority: 700
# EOF

# # Install chromium and chromium-driver
# !apt-get update
# !apt-get install chromium chromium-driver

# # Install Selenium
# !pip install selenium

%%shell
# Install chromium and chromium-driver using apt
# apt-get update
# apt-get install chromium chromium-driver # original code

# Suggested Change: Install chromium using wget
# Debian no longer provides chromium in Buster or Bullseye
# Proposed solution: https://gist.github.com/korakot/f908ecc86f265ea290f5c6a9c7524031
!apt-get update  # Update package lists to ensure apt can find the necessary packages
!apt-get install -y wget curl unzip # Install wget, curl, and unzip if they are not already present

wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
apt install ./google-chrome-stable_current_amd64.deb -y

# Install chromedriver using wget
# Proposed solution: https://stackoverflow.com/a/74827505
CHROMEDRIVER_VERSION=$(curl -sS -o - https://chromedriver.storage.googleapis.com/LATEST_RELEASE)
wget -q --continue https://chromedriver.storage.googleapis.com/${CHROMEDRIVER_VERSION}/chromedriver_linux64.zip
unzip chromedriver_linux64.zip
chmod +x chromedriver
mv chromedriver /usr/bin/chromedriver

# Install Selenium
!pip install selenium

/bin/bash: line 8: !apt-get: command not found
/bin/bash: line 9: !apt-get: command not found
--2025-03-30 16:24:24--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.251.2.136, 142.251.2.93, 142.251.2.190, ...
Connecting to dl.google.com (dl.google.com)|142.251.2.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114483440 (109M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 109.18M   225MB/s    in 0.5s    

2025-03-30 16:24:24 (225 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [114483440/114483440]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers
The following NEW packa

CalledProcessError: Command '# Install chromium and chromium-driver using apt
# apt-get update
# apt-get install chromium chromium-driver # original code

# Suggested Change: Install chromium using wget
# Debian no longer provides chromium in Buster or Bullseye
# Proposed solution: https://gist.github.com/korakot/f908ecc86f265ea290f5c6a9c7524031
!apt-get update  # Update package lists to ensure apt can find the necessary packages
!apt-get install -y wget curl unzip # Install wget, curl, and unzip if they are not already present

wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
apt install ./google-chrome-stable_current_amd64.deb -y

# Install chromedriver using wget
# Proposed solution: https://stackoverflow.com/a/74827505
CHROMEDRIVER_VERSION=$(curl -sS -o - https://chromedriver.storage.googleapis.com/LATEST_RELEASE)
wget -q --continue https://chromedriver.storage.googleapis.com/${CHROMEDRIVER_VERSION}/chromedriver_linux64.zip
unzip chromedriver_linux64.z

In [ ]:
import os
import requests
import time
import json

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
WEBDRIVER_DELAY_TIME_INT = 10
TIMEOUT_INT = 10

# Initialize the Chrome WebDriver Service
service = Service(executable_path=r"/usr/bin/chromedriver")

# Set Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("window-size=1920x1080")
chrome_options.headless = True

# Initialize the WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.implicitly_wait(TIMEOUT_INT)  # Set implicit wait time

# Initialize explicit wait
wait = WebDriverWait(driver, WEBDRIVER_DELAY_TIME_INT)

In [ ]:
def extract_audio_links_from_menu(menu_url, driver):
    # Navigate to the menu URL
    driver.get(menu_url)

    # Locate the container holding the play items
    container = wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "div.w-full.flex.flex-col.gap-3.pt-3")
    ))

    # Find all play items within the container
    play_items = container.find_elements(By.CSS_SELECTOR, "div.play-item")

    # Extract links from the play items
    links = []
    for item in play_items:
        try:
            a_tag = item.find_element(By.CSS_SELECTOR, ".ptxt-track a")
            link = a_tag.get_attribute("href")
            links.append(link)
        except Exception:
            continue

    return links

In [ ]:
def download_audio_file(file_url, filepath):
    # Send a GET request to download the file
    response = requests.get(file_url, stream=True)

    if response.status_code == 200:
        # Write the file content in chunks to avoid memory issues
        with open(filepath, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
    else:
        # Handle errors if the file cannot be downloaded
        print(f"Error downloading file from {file_url}; status: {response.status_code}")

In [ ]:
def extract_track_info(driver):
    # Extract track information from the page
    audio_div = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-track-info]"))
    )
    return json.loads(audio_div.get_attribute("data-track-info"))

def extract_genres(driver):
    # Extract genres from the page
    try:
        genre_elem = driver.find_element(By.CSS_SELECTOR, "span.md\\:col-span-6.flex.flex-wrap.gap-3")
        return [a.text.strip() for a in genre_elem.find_elements(By.TAG_NAME, "a") if a.text.strip()]
    except Exception:
        return []

def extract_duration(driver):
    # Extract the duration of the track
    try:
        duration_elem = driver.find_element(By.CSS_SELECTOR,
                                            "span.w-12.ml-auto.md\\:ml-0.col-span-2.inline-flex.justify-end.items-center")
        return duration_elem.text.strip()
    except Exception:
        return ""

In [ ]:
def extract_extra_info(driver):
    # Extract additional information like 'Instrumental' and 'AI generated?'
    instrumental = "No"
    ai_generated = "No"
    try:
        info_container = driver.find_element(By.CSS_SELECTOR,
                                             "div.px-8.py-2.bg-gray-light.flex.flex-col.divide-y.divide-gray")
        info_divs = info_container.find_elements(By.CSS_SELECTOR,
                                                 "div.grid.grid-cols-1.md\\:grid-cons-8.py-6")
        for div in info_divs:
            label = div.find_element(By.CSS_SELECTOR, "span.font-\

\[500\\]

.md\\:col-span-2").text.strip()
            value = div.find_element(By.CSS_SELECTOR, "span.md\\:col-span-6").text.strip()
            if "Instrumental" in label:
                instrumental = value
            if "AI generated?" in label:
                ai_generated = value
    except Exception:
        pass
    return instrumental, ai_generated

In [ ]:
def process_audio_page(audio_url, driver, index):
    # Process individual audio page and save metadata
    driver.get(audio_url)

    track_info = extract_track_info(driver)
    file_url = track_info.get("fileUrl", "")
    audio_name = track_info.get("title", "").strip()
    author = track_info.get("artistName", "").strip()
    genres = extract_genres(driver)
    duration = extract_duration(driver)
    instrumental, ai_generated = extract_extra_info(driver)

    metadata = {
        "audioName": audio_name,
        "author": author,
        "genres": genres,
        "instrumental": instrumental,
        "ai_generated": ai_generated,
        "duration": duration,
        "audio_url": audio_url
    }

    audio_filename = f"audio_{index:04d}.mp3"
    meta_filename = f"audio_{index:04d}.json"
    audio_filepath = os.path.join("crawled_data", "audio", audio_filename)
    meta_filepath = os.path.join("crawled_data", meta_filename)

    download_audio_file(file_url, audio_filepath)
    with open(meta_filepath, "w", encoding="utf-8") as f:
        json.dump(metadata, f, ensure_ascii=False, indent=4)

    return metadata

In [ ]:
def loop_over_menu_pages(base_url, total_pages, driver):
    all_links = []

    for page in tqdm(range(1, total_pages + 1), desc="Extracting Links", unit="page"):
        page_url = f"{base_url}?page={page}"
        try:
            # Extract links from the current page and add to the list
            links = extract_audio_links_from_menu(page_url, driver)
            all_links.extend(links)
        except Exception as e:
            # Handle errors gracefully
            print(f"Error on page {page}: {e}")

    return all_links

In [ ]:
# Create necessary directories
os.makedirs("crawled_data", exist_ok=True)
os.makedirs(os.path.join("crawled_data", "audio"), exist_ok=True)

# Define base URL and parameters
base_url = "https://freemusicarchive.org/genre/piano/"
total_pages = 10
sample_idx = 1

# Extract audio links from menu pages
audio_links = loop_over_menu_pages(base_url, total_pages, driver)
print(f"Total audio links extracted: {len(audio_links)}")

# Download audio and metadata
for audio_url in tqdm(audio_links, desc="Downloading Audio and Metadata", unit="audio"):
    try:
        process_audio_page(audio_url, driver, sample_idx)
        sample_idx += 1
    except Exception as e:
        print(f"Error at: {audio_url} - {e}")
        continue
    time.sleep(0.5)

# Close the WebDriver
driver.quit()

In [ ]:
!pip install numba==0.61.0 torchaudio==2.6.0 librosa==0.10.2.post1

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
import torchaudio.transforms as T
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd
import torch.nn.functional as F
import matplotlib.pyplot as plt
import json

from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from IPython.display import Audio
from tqdm import tqdm

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Download the file using gdown
!gdown 1u2WzsWUlyZbbPDfXAWXuLRMTwFkT21wa

# Unzip the downloaded file
!unzip -q crawled_piano_audio_40_pages.zip -d piano_audio_files_40_pages

# Remove the zip file after extraction
os.remove("crawled_piano_audio_40_pages.zip")

In [ ]:
def load_and_get_genres(json_path):
    # Load JSON and extract genres
    with open(json_path, "r") as f:
        data = json.load(f)
    return data.get('genres', [])

def load_and_resample_audio(file_path, target_sr=22050):
    # Load and resample audio to the target sample rate
    audio, sr = librosa.load(file_path, sr=None)
    if sr != target_sr:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)
    return audio, target_sr

def audio_to_melspec(audio, sr, n_mels, n_fft=2048, hop_length=512, to_db=False):
    # Convert audio to Mel Spectrogram
    spec = librosa.feature.melspectrogram(
        y=audio,
        sr=sr,
        n_fft=n_fft,
        hop_length=hop_length,
        win_length=None,
        window="hann",
        center=True,
        pad_mode="reflect",
        power=2.0,
        n_mels=n_mels
    )
    if to_db:
        spec = librosa.power_to_db(spec, ref=np.max)
    return spec

def normalize_melspec(melspec, norm_range=(0, 1)):
    # Normalize Mel Spectrogram
    scaler = MinMaxScaler(feature_range=norm_range)
    melspec = melspec.T
    melspec_normalized = scaler.fit_transform(melspec)
    return melspec_normalized.T

def denormalize_melspec(melspec_normalized, original_melspec, norm_range=(0, 1)):
    # Denormalize Mel Spectrogram
    scaler = MinMaxScaler(feature_range=norm_range)
    melspec = original_melspec.T
    scaler.fit(melspec)
    melspec_denormalized = scaler.inverse_transform(melspec_normalized.T)
    return melspec_denormalized.T

def melspec_to_audio(melspec, sr, n_fft=2048, hop_length=512, n_iter=64):
    # Convert Mel Spectrogram back to audio
    if np.any(melspec < 0):
        melspec = librosa.db_to_power(melspec)
    audio_reconstructed = librosa.feature.inverse.mel_to_audio(
        melspec,
        sr=sr,
        n_fft=n_fft,
        hop_length=hop_length,
        win_length=None,
        window="hann",
        center=True,
        pad_mode="reflect",
        power=2.0,
        n_iter=n_iter
    )
    return audio_reconstructed

def display_audio_files(reconstructed_audio, sr, title="", original_audio=None):
    # Display audio files
    if original_audio is not None:
        print("Original Audio:")
        ipd.display(ipd.Audio(original_audio, rate=sr))
        print("Reconstructed Audio (from Mel Spectrogram):")
    else:
        print(title)
    ipd.display(ipd.Audio(reconstructed_audio, rate=sr))

def show_spectrogram(spectrogram, title="Mel-Spectrogram", denormalize=False, is_numpy=False):
    # Show Mel Spectrogram
    if not is_numpy:
        spectrogram = spectrogram.squeeze().cpu().numpy()
    plt.figure(figsize=(10, 4))
    if denormalize:
        plt.imshow(spectrogram, aspect="auto", origin="lower", cmap="viridis")
    else:
        plt.imshow(spectrogram, aspect="auto", origin="lower", cmap="viridis", vmin=0, vmax=1)
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Mel Frequency")
    plt.colorbar()
    plt.show()

In [ ]:
# Define the path to the directory containing JSON files
json_dir = os.path.join("piano_audio_files", "crawled_data")
all_genres = []

# Iterate through all JSON files in the directory
for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        # Load genres from each JSON file
        json_path = os.path.join(json_dir, filename)
        genres = load_and_get_genres(json_path)
        all_genres.extend(genres)

# Extract unique genres and display results
unique_genres = set(all_genres)
max_genres = len(unique_genres)

print(f"Total unique genres: {max_genres}")
print(f"Unique genres: {unique_genres}")

In [ ]:
# Map genres to indices and vice versa
genres2idx = {genre: idx for idx, genre in enumerate(unique_genres)}
idx2genres = {idx: genre for genre, idx in genres2idx.items()}

# Tokenize genres into their corresponding indices
def tokenize(genres):
    return [genres2idx[genre] for genre in genres if genre in genres2idx]

# Detokenize tokens back into their corresponding genres
def detokenize_tolist(tokens):
    return [idx2genres[token] for token in tokens if token in idx2genres]

# One-hot encode the list of tokenized genres
def onehot_encode(tokens, max_genres):
    onehot = np.zeros(max_genres)
    onehot[tokens] = 1
    return onehot

# Decode one-hot encoded data back into a list of genre indices
def onehot_decode(onehot):
    return [idx for idx, val in enumerate(onehot) if val == 1]

In [ ]:
class AudioDataset(Dataset):
    def __init__(self, data_dir, json_dir, sample_rate, duration, n_mels, n_genres, testset_amount=10):
        self.data_dir = data_dir
        self.files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".mp3")]
        self.json_dir = json_dir
        self.json_files = [os.path.join(json_dir, f) for f in os.listdir(json_dir) if f.endswith(".json")]
        self.sample_rate = sample_rate
        self.duration = duration
        self.fixed_length = sample_rate * duration
        self.n_genres = n_genres
        self.n_mels = n_mels
        audios = []

        # Load audio files and their metadata
        for file_path, json_file_path in tqdm(zip(self.files, self.json_files),
                                              desc=f"Loading audio files in {data_dir}",
                                              unit="file", total=len(self.files)):
            audio, sr = load_and_resample_audio(file_path, target_sr=sample_rate)
            genres_list = load_and_get_genres(json_file_path)
            genres_tokens = tokenize(genres_list)
            genres_input = onehot_encode(genres_tokens, n_genres)
            genres_input = torch.tensor(genres_input, dtype=torch.long).unsqueeze(0)

            n_samples = len(audio)
            n_segments = n_samples // self.fixed_length

            for i in range(n_segments):
                start = i * self.fixed_length
                end = (i + 1) * self.fixed_length
                segment = audio[start:end]
                mel_spec = audio_to_melspec(segment, sr, self.n_mels, to_db=True)
                mel_spec_norm = normalize_melspec(mel_spec)
                mel_spec = torch.tensor(mel_spec, dtype=torch.float32).unsqueeze(0)
                mel_spec_norm = torch.tensor(mel_spec_norm, dtype=torch.float32).unsqueeze(0)
                audios.append((mel_spec_norm, genres_input, mel_spec))

        # Split the dataset into training and test sets
        self.audios = audios[:len(audios) - testset_amount]
        self.testset = audios[len(audios) - testset_amount:]

        print(f"Loaded {len(self.audios)} audio segments from {len(self.files)} files, "
              f"each with shape: {self.audios[0][0].shape}, {self.audios[0][1].shape}, "
              f"duration: {duration} seconds")
        print(f"Test set: {len(self.testset)} audio segments")

    def __len__(self):
        return len(self.audios)

    def __getitem__(self, idx):
        mel_spec_part, genres_input, mel_spec = self.audios[idx]
        return mel_spec_part, genres_input, mel_spec

In [ ]:
# Define parameters
sample_rate = 22050
duration = 3  # Duration of audio segments in seconds
n_mels = 256  # Number of Mel spectrogram bins
batch_size = 128

# Define directories
audio_dir = os.path.join("piano_audio_files", "crawled_data", "audio")
json_dir = os.path.join("piano_audio_files", "crawled_data")

# Amount of test data
testset_amount = 32

# Load the dataset
trainset = AudioDataset(audio_dir, json_dir, sample_rate, duration, n_mels, max_genres, testset_amount=testset_amount)
testset = trainset.testset

# Check if training data exists
if len(trainset) == 0:
    raise ValueError(f"No .wav file found in {audio_dir}.")

# Create DataLoaders for training and testing
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=testset_amount, shuffle=False, num_workers=2)

In [ ]:
class CVAE(nn.Module):
    def __init__(self, d_model, latent_dim, n_frames, n_mels, n_genres):
        super(CVAE, self).__init__()
        self.d_model = d_model
        self.latent_dim = latent_dim
        self.n_frames = int(np.ceil(n_frames / 2**3))
        self.n_mels = int(np.ceil(n_mels / 2**3))
        self.n_genres = n_genres

        print(self.n_frames, self.n_mels)

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1 + self.n_genres, d_model, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(d_model),
            nn.SiLU(),
            nn.Dropout2d(0.05),

            nn.Conv2d(d_model, d_model * 2, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(d_model * 2),
            nn.SiLU(),
            nn.Dropout2d(0.1),

            nn.Conv2d(d_model * 2, d_model * 4, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(d_model * 4),
            nn.SiLU(),
            nn.Dropout2d(0.15),

            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten()
        )

        # Latent space
        self.fc_mu = nn.Linear(d_model * 4, latent_dim)
        self.fc_logvar = nn.Linear(d_model * 4, latent_dim)

        # Decoder
        self.decoder_input = nn.Linear(latent_dim + self.n_genres, d_model * 4 * self.n_frames * self.n_mels)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(d_model * 4, d_model * 2, kernel_size=3, stride=2, padding=1, output_padding=(1, 0)),
            nn.BatchNorm2d(d_model * 2),
            nn.SiLU(),
            nn.Dropout2d(0.1),

            nn.ConvTranspose2d(d_model * 2, d_model, kernel_size=3, stride=2, padding=1, output_padding=(1, 0)),
            nn.BatchNorm2d(d_model),
            nn.SiLU(),
            nn.Dropout2d(0.05),

            nn.ConvTranspose2d(d_model, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x, genres_input):
        ori_genres_embed = genres_input.view(genres_input.size(0), -1)
        genres_embed = ori_genres_embed.unsqueeze(-1).unsqueeze(-1)
        genres_embed = genres_embed.expand(-1, -1, x.size(2), x.size(3))
        x_genres = torch.cat((x, genres_embed), dim=1)

        h = x_genres
        shortcuts = []
        for block in self.encoder:
            h = block(h)
            if isinstance(block, nn.SiLU):
                shortcuts.append(h)

        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)

        z = self.reparameterize(mu, logvar)
        z_genres = torch.cat((z, ori_genres_embed), dim=1)

        h_dec = self.decoder_input(z_genres)
        h_dec = h_dec.view(-1, self.d_model * 4, self.n_frames, self.n_mels)

        for block in self.decoder:
            if isinstance(block, nn.ConvTranspose2d) and shortcuts:
                shortcut = shortcuts.pop()
                h_dec = h_dec + shortcut
            h_dec = block(h_dec)

        recon = h_dec[:, :, :x.size(2), :x.size(3)]
        return recon, mu, logvar

In [ ]:
def loss_function(recon_x, x, mu, logvar):
    # Calculate reconstruction loss
    recon_loss = nn.functional.mse_loss(recon_x, x, reduction="sum")

    # Calculate KL divergence
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # Return the total loss
    return recon_loss + KLD


def train_vae(model, dataloader, optimizer, scheduler, num_epochs, verbose_interval=50):
    model.train()  # Set the model to training mode
    losses = []

    for epoch in tqdm(range(num_epochs), desc="Training", unit="epoch"):
        train_loss = 0

        for batch_idx, (data, genres_input, ori_data) in enumerate(dataloader):
            data = data.to(device)
            genres_input = genres_input.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            recon, mu, logvar = model(data, genres_input)

            # Compute loss and backpropagate
            loss = loss_function(recon, data, mu, logvar)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()

        # Update the learning rate scheduler
        scheduler.step()

        # Compute average loss for the epoch
        avg_loss = train_loss / len(dataloader.dataset)
        losses.append(avg_loss)

        # Print epoch summary
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}, "
              f"Lr: {scheduler.get_last_lr()[0]}")

        # Visualize spectrograms at specified intervals
        if epoch == 0 or (epoch + 1) % verbose_interval == 0:
            data = data[0].detach().cpu()
            recon_img = recon[0].detach().cpu()
            show_spectrogram(data, title="Original Spectrogram")
            show_spectrogram(recon_img, title="Reconstructed Spectrogram")

    return mu, logvar, losses

In [ ]:
# Define parameters
d_model = 64
latent_dim = 128
lr = 2e-4
num_epochs = 100
step_size = num_epochs // 2
verbose_interval = num_epochs // 10
gamma = 0.5

# Initialize the CVAE model
model = CVAE(d_model, latent_dim, n_mels, frame, max_genres).to(device)

# Define optimizer and learning rate scheduler
optimizer = optim.AdamW(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

# Print total number of parameters in the model
print(f"Total number of parameters: {sum(p.numel() for p in model.parameters())}")

# Train the model
mu, logvar, losses = train_vae(model, trainloader, optimizer, scheduler, num_epochs, verbose_interval=verbose_interval)

In [ ]:
def generate(model, dataloader, genres_list, num_samples=5, diff_level=1):
    model.eval()  # Set model to evaluation mode

    with torch.no_grad():
        # Get a batch of data from the dataloader
        data, old_genres_input, ori_data = next(iter(dataloader))
        data = data.to(device)

        # Prepare genres input
        genres_tokens = tokenize(genres_list)
        genres_input = onehot_encode(genres_tokens, model.n_genres)
        genres_input = torch.tensor(genres_input, dtype=torch.long).unsqueeze(0)
        genres_input = genres_input.repeat(old_genres_input.shape[0], 1)
        genres_input = genres_input.to(device)

        # Generate reconstructed data
        recon, mu, logvar = model(data, genres_input)

        ori_audios = []
        recon_audios = []

        for i in range(num_samples):
            # Get genres for original data
            old_genres_list = detokenize_tolist(onehot_decode(old_genres_input[i].squeeze().tolist()))

            # Display spectrograms
            show_spectrogram(data[i], title="Original Spectrogram with Genres: " + ", ".join(old_genres_list))
            show_spectrogram(recon[i], title="Reconstructed Spectrogram with Genres: " + ", ".join(genres_list))

            # Compute and display difference spectrogram
            diff_spectrogram = torch.abs(data[i] - recon[i]) * diff_level
            show_spectrogram(diff_spectrogram, title=f"Difference Spectrogram (|Original - Reconstructed|) * {diff_level}")

            # Print loss
            print("Loss: ", loss_function(recon[i], data[i], mu, logvar).item())

            # Reconstruct audio from spectrogram
            spec_denorm = denormalize_melspec(recon[i].cpu().numpy().squeeze(), ori_data[i].cpu().numpy().squeeze())
            audio_reconstructed = melspec_to_audio(spec_denorm, sr=sample_rate)
            ori_audio = melspec_to_audio(ori_data[i].cpu().numpy().squeeze(), sr=sample_rate)

            # Store audio results
            recon_audios.append(audio_reconstructed)
            ori_audios.append(ori_audio)

            # Display audio files
            display_audio_files(ori_audio, sample_rate, title="Original Audio with Genres: " + ", ".join(old_genres_list))
            display_audio_files(audio_reconstructed, sample_rate, title="Reconstructed Audio with Genres: " + ", ".join(genres_list))

        # Concatenate audio samples if num_samples > 1
        if num_samples > 1:
            print("-" * 100, "Connect all audio", "-" * 100)
            recon_ori_audios = np.concatenate(ori_audios)
            display_audio_files(recon_ori_audios, sample_rate, title="Connect all original audio")
            recon_audios = np.concatenate(recon_audios)
            display_audio_files(recon_audios, sample_rate, title="Connect all reconstructed audio")